# 11q deletion syndrome

11q deletion syndrome (Jacobsen syndrome) is a  contiguous gene syndrome caused by partial deletion of the long arm of chromosome 11 whose size ranges from approximately 7 to 20 Mb, with the proximal breakpoint within or telomeric to subband 11q23.3 and the deletion extending usually to the telomere.

The data in this notebook is dervied from [Grossfeld PD, et al. (2004) The 11q terminal deletion disorder: a prospective study of 110 cases. Am J Med Genet A. 129A:51-61](https://pubmed.ncbi.nlm.nih.gov/15266616/).

Note individual 133 was removed owing to lack of data.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import HTML, display
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.12


In [5]:
PMID = "PMID:15266616"
title = "The 11q terminal deletion disorder: a prospective study of 110 cases"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser("../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


## Data input

In [6]:
df = pd.read_csv("/Users/robinp/GIT/phenopacket-store/notebooks/11q_terminal_deletion/input/grossfield2004_supplement.txt", sep='\t')
df["individual_id"] = df["individual_id"].astype(int)
df = df.astype(str)
df.head()

,individual_id,asymmetrical skull/craniosynostenosis,trigonocephaly,metopic ridge,flat occiput,prominent occiput,macrocephaly,microcephaly,High forehead,short forehead,...,ureteral duplication,urinary reflux/funzione renale,genital anomalies,recurrent infectionts,immunological problems,tumors,low/abnormal platelets,behaviour problems,seizures/EEG anomalie,obesità
0,83,1<,1>,0,0,0,0.0,0.0,1,0,...,1.0,1,0,1.0,0.0,0.0,1,0,0.0,nan
1,17,1,0,0,0,0,1.0,0.0,0,1,...,0.0,0,0,1.0,0.0,0.0,0,0,0.0,nan
2,108,0,0,0,0,0,0.0,1.0,0,1,...,0.0,0,1,1.0,0.0,0.0,1,1,1.0,0
3,34,1,1,0,0,1>,0.0,1.0,1,0,...,0.0,1,0,0.0,0.0,0.0,1,0,1.0,0
4,80,0,0,0,0,0,1.0,0.0,1,0,...,0.0,0,1,0.0,0.0,1.0,1,1,0.0,nan


## Processing data
The following codes are used in some of the data cells

- <: feature disapperead during follow up
- &gt;: feature apperead during follow up
- -: feature disapperead after surgery
- +: feature apperead after surgery
- =: feature still present after surgery
- *: ancient photos

Since these are not relevant for parsing, we will remove all such codes.

In [7]:
import re
def remove_redundant(cell_contents):
    res, n = re.subn('[><\-+=*]', '', cell_contents)
    return res
df = df.applymap(remove_redundant)

In [8]:
generator = SimpleColumnMapperGenerator(df=df, observed="1", excluded="0",hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()

In [9]:
display(HTML(generator.to_html()))

Result,Columns
Mapped,trigonocephaly; flat occiput; prominent occiput; macrocephaly; microcephaly; High forehead; prominent forehead; wide forehead; narrow forehead; flat face; triangular face; round face; broad face; hypertelorism; epicanthic folds; thick eyebrows; sparse eyebrows; arched eyebrows; prominent eyebrows; long eyelashes; downslanted palpebral fissures; ptosis; deep-set eyes; proptosis; iris coloboma; cataract; lens dislocation; convergent strabismus; divergent strabismus; wide nasal bridge; long nose; upturned nasal tip; bulbous nasal tip; anteverted nares; small nasal tip; prominent columella; long philtrum; short philtrum; smooth philtrum; deep philtrum; small mouth; thin upper lip; thick upper lip; thin lower lip; thick lower lip; everted lower lip; high palate; cleft palate; dental anomalies; abnormally shaped teeth; full cheeks; pointed chin; prognathism; small ears; prominent ears; low set ears; posteriorly rotated ears; conductive deafness; low frontal hairline; brittle hair; short neck; webbed neck; thenar hypoplasia; hypothenar hypoplasia; broad thumbs; thin fingers; long fingers; flat feet; club feet; short toes; joint laxity; inguinal hernia; umbilical hernia; skeletal anomalies; constipation; ureteral duplication; genital anomalies
Unmapped,"individual_id; asymmetrical skull/craniosynostenosis; metopic ridge; short forehead; straight forehead; thin/long face; asymmetric face; wide eyebrows; narrow eyebrows; lateral hypoplasia of eyebrows; triangular eyebrows; synophria; upslanted palpebral fissures (inserito dopo); eyelids ectropion/Kabuki; eyelids coloboma; thickened lids; large eye s; cornea anomalies; pigmentary anomalies of iris; other (iris); retinal vessel anomalies; retina anomalies; not specified strabismus; vision abnormalities; nasolacrimal duct abnormalities; wide nasal root; flat nasal root; prominent nasal root; thin nasal bridge; flat nasal bridge; prominent nasal bridge; short nose; lateral nares; broad nares; small nares; absent/short columella; upturned philtrum; V mouth shape; down-turned corners of the mouth (carp mouth); scialorrhea; flat cheek; flat malar region; micro/ retrognathia; over-folded helix; triangular helix; straight helix; round helix; fleshy helix; thin helix; trago hypoplasia; hypoplastic lobule; uplifted ear lobule; neurosensorial deafness; not specified deafness; preauricolar fistula; prreauricolar sinus; preauricolar tags; low posterior hairline ; large neck; Aural duct stenosis; age; sex; individual_id_hands; wide palm hands; increased ratio palm/fingers; low set thumbs; flat thumbs; short fingers; fingers clinodactyly; clinodactyly V finger; short V finger; large I interphalangeal joint; thin II interphalangeal joint; long I phalange; long II phalange; large III phalange; thin III phalange; long III phalange; short III phalange; convex nails; distal nails; flat finger pads; fetal finger pads; palmar crease anomalies; fingers camptodactyly; skin sindactyly; individual_id_feet; stubby feet; long I toe; large I toe; short I toe; valgus I toe; toes clinodactyly; toes 2,3 syndactyly; toes 2=3; crowded toes; individual_id.1; joint contractures; chest/polmone anomalies; jatal hernia; sacral dimple/angioma; voice anomalies; skin anomalies; scoliosis/cyphosis/lordosi; brain anomalies; hormon anomalies; heart malformation; stenosis aortic or pulmonary; pyloric stenosis aggiunta; other gastrointestinal anomalies; kidney duplication; horseshoe kidney /abnormal shape; hypoplastic/agenesis kidney; urinary reflux/funzione renale; recurrent infectionts; immunological problems; tumors; low/abnormal platelets; behaviour problems; seizures/EEG anomalie; obesità"


In [10]:
# Note that  upturned nasal tip is mapped to Anteverted nares in HPO; some of the entires contradict;
# Similarly, prominent eyebrows is a synonym with Thick eyeborw in the HPO but distinct here.
# therefore we remove it
column_mapper_d.pop("upturned nasal tip");
column_mapper_d.pop("prominent eyebrows");

In [11]:
#result = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr, omit_columns=generator.get_mapped_columns())
# result was used to help create the following
simple_d = {
    "metopic ridge": ["Prominent metopic ridge", "HP:0005487"],
    "short forehead": ["Small forehead", "HP:0000350"],
    "thin/long  face": ["Long face", "HP:0000276"],
    "asymmetric face":["Facial asymmetry", "HP:0000324"],
    "wide eyebrows": ["Broad eyebrow", "HP:0011229"],
    "lateral hypoplasia of  eyebrows":["Sparse lateral eyebrow", "HP:0005338"],
    "synophria":["Synophrys", "HP:0000664"],
    "upslanted palpebral fissures (inserito dopo)": ["Upslanted palpebral fissure", "HP:0000582"],
    "eyelids ectropion/Kabuki":["Ectropion", "HP:0000656"],
    "eyelids coloboma":["Eyelid coloboma", "HP:0000625"],
    "thickened lids":["Palpebral thickening", "HP:0030939"],
    "large eye s": ["Abnormally large globe", "HP:0001090"],
    "retinal vessel anomalies": ["Abnormal retinal vascular morphology", "HP:0008046"],
    "not specified strabismus": ["Strabismus", "HP:0000486"],
    "wide nasal root": ["Wide nasal bridge", "HP:0000431"],
    "flat  nasal bridge": ["Depressed nasal bridge", "HP:0005280"],
    "prominent  nasal bridge": ["Prominent nasal bridge", "HP:0000426"],
    "thin nasal bridge": ["Narrow nasal bridge", "HP:0000446"],
    "short  nose": ["Short nose", "HP:0003196"],
    "small nares": ["Short nose", "HP:0003196"],
    "broad nares": ["Wide nose", "HP:0000445"],
    "absent/short columella": ["Short columella", "HP:0002000"],
    "down-turned corners of the mouth (carp mouth)": ["Downturned corners of mouth", "HP:0002714"],
    "flat malar region": ["Malar flattening", "HP:0000272"],
    "micro/ retrognathia": ["Micrognathia", "HP:0000347"],
    "over-folded helix": ["Overfolded helix", "HP:0000396"],
    "thin helix": ["Thin ear helix", "HP:0009905"],
    "trago hypoplasia": ["Underdeveloped tragus", "HP:0011272"],
    "hypoplastic lobule": ["Small earlobe", "HP:0000385"],
    "uplifted ear lobule": ["Uplifted earlobe", "HP:0009909"],
    "neurosensorial deafness": ["Sensorineural hearing impairment", "HP:0000407"],
    "not specified deafness": ["Hearing impairment", "HP:0000365"],
    "prreauricolar sinus": ["Preauricular pit", "HP:0004467"],
    "low posterior hairline ": ["Low posterior hairline", "HP:0002162"],
    "long I toe": ["Long hallux", "HP:0001847"],
    "valgus I toe": ["Hallux valgus", "HP:0001822"],
    "toes clinodactyly": ["Toe clinodactyly", "HP:0001863"],
    "toes 2,3 syndactyly": ["2-3 toe syndactyly", "HP:0004691"],
    "wide palm hands": ["Broad palm", "HP:0001169"],
    "short fingers": ["Short finger", "HP:0009381"],
    "fingers clinodactyly": ["Finger clinodactyly", "HP:0040019"],
    "clinodactyly V finger": ["Clinodactyly of the 5th finger", "HP:0004209"],
    "convex nails": ["Hyperconvex fingernails", "HP:0001812"],
    "fetal finger pads": ["Prominent fingertip pads", "HP:0001212"],
    "fingers camptodactyly": ["Camptodactyly of finger", "HP:0100490"],
    "joint contractures": ["Joint contracture", "HP:0034392"],
    "jatal hernia": ["Hiatus hernia", "HP:0002036"],
    "sacral dimple/angioma": ["Sacral dimple", "HP:0000960"],
    "kidney duplication": ["Renal duplication", "HP:0000075"],
    "low/abnormal platelets": ["Thrombocytopenia", "HP:0001873"],
    "behaviour problems": ["Atypical behavior", "HP:0000708"],
    "seizures/EEG anomalie": ["Seizure", "HP:0001250"],
    "obesità": ["Obesity", "HP:0001513"],
}
for k, v in simple_d.items():
    column_mapper_d[k] = SimpleColumnMapper(hpo_id=v[1], hpo_label=v[0], observed="Y", excluded="N")

In [12]:
ageColumn = AgeColumnMapper.iso8601(column_name="age")
sexColumn = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='sex')

In [13]:
encoder = CohortEncoder(df=df,
                        hpo_cr=hpo_cr,
                        column_mapper_d=column_mapper_d,
                        individual_column_name="individual_id",
                        agemapper=ageColumn,
                        sexmapper=sexColumn,
                        metadata=metadata)
jacobsen = Disease(disease_id="OMIM:147791", disease_label="Jacobsen syndrome")
encoder.set_disease(jacobsen)

In [14]:
import phenopackets as PPKt
import random
import string

def get_variant():
    vdescriptor = PPKt.VariationDescriptor()
    vdescriptor.id =  "var_" + "".join(random.choices(string.ascii_letters, k=25))
    vdescriptor.label = "telomeric 11q deletion"
    vdescriptor.molecule_context = PPKt.MoleculeContext.genomic
    vdescriptor.allelic_state.id = "GENO:0000135"
    vdescriptor.allelic_state.label = "heterozygous"
    vdescriptor.structural_type.id = "SO:1000029"
    vdescriptor.structural_type.label = "chromosomal_deletion"
    vinterpretation = PPKt.VariantInterpretation()
    vinterpretation.variation_descriptor.CopyFrom(vdescriptor)
    return vinterpretation

In [15]:
individuals = encoder.get_individuals()
for i in individuals:
    chr11del = get_variant()
    i.add_variant(chr11del)

Could not parse the following as ISO8601 ages: nan (n=36)


In [16]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,58
INFORMATION,NOT_MEASURED,4576


In [17]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
83 (FEMALE; ),Jacobsen syndrome (OMIM:147791),telomeric 11q deletion: chromosomal_deletion (SO:1000029),Trigonocephaly (HP:0000243); High forehead (HP:0000348); Prominent forehead (HP:0011220); Narrow forehead (HP:0000341); Triangular face (HP:0000325); Hypertelorism (HP:0000316); Epicanthus (HP:0000286); Sparse eyebrow (HP:0045075); Highly arched eyebrow (HP:0002553); Long eyelashes (HP:0000527); Downslanted palpebral fissures (HP:0000494); Ptosis (HP:0000508); Esodeviation (HP:0020045); Wide nasal bridge (HP:0000431); Anteverted nares (HP:0000463); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Full cheeks (HP:0000293); Posteriorly rotated ears (HP:0000358); Brittle hair (HP:0002299); Pes planus (HP:0001763); excluded: Flat occiput (HP:0005469); excluded: Prominent occiput (HP:0000269); excluded: Broad forehead (HP:0000337); excluded: Flat face (HP:0012368); excluded: Round face (HP:0000311); excluded: Broad face (HP:0000283); excluded: Thick eyebrow (HP:0000574); excluded: Deeply set eye (HP:0000490); excluded: Proptosis (HP:0000520); excluded: Iris coloboma (HP:0000612); excluded: Cataract (HP:0000518); excluded: Ectopia lentis (HP:0001083); excluded: Exodeviation (HP:0020049); excluded: Long nose (HP:0003189); excluded: Bulbous nose (HP:0000414); excluded: Hypoplastic nasal tip (HP:0005278); excluded: Low hanging columella (HP:0009765); excluded: Short philtrum (HP:0000322); excluded: Deep philtrum (HP:0002002); excluded: Thick upper lip vermilion (HP:0000215); excluded: Thin lower lip vermilion (HP:0010282); excluded: Thick lower lip vermilion (HP:0000179); excluded: Everted lower lip vermilion (HP:0000232); excluded: High palate (HP:0000218); excluded: Cleft palate (HP:0000175); excluded: Abnormal dental morphology (HP:0006482); excluded: Microtia (HP:0008551); excluded: Low-set ears (HP:0000369); excluded: Conductive hearing impairment (HP:0000405); excluded: Low anterior hairline (HP:0000294); excluded: Short neck (HP:0000470); excluded: Webbed neck (HP:0000465); excluded: Talipes equinovarus (HP:0001762); excluded: Umbilical hernia (HP:0001537); excluded: Abnormality of the genital system (HP:0000078)
17 (FEMALE; P6Y5M),Jacobsen syndrome (OMIM:147791),telomeric 11q deletion: chromosomal_deletion (SO:1000029),Prominent forehead (HP:0011220); Broad forehead (HP:0000337); Triangular face (HP:0000325); Hypertelorism (HP:0000316); Epicanthus (HP:0000286); Thick eyebrow (HP:0000574); Highly arched eyebrow (HP:0002553); Downslanted palpebral fissures (HP:0000494); Ptosis (HP:0000508); Exodeviation (HP:0020049); Wide nasal bridge (HP:0000431); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Thin lower lip vermilion (HP:0010282); Abnormality of the dentition (HP:0000164); Microtia (HP:0008551); Low anterior hairline (HP:0000294); Short neck (HP:0000470); Webbed neck (HP:0000465); Small thenar eminence (HP:0001245); Broad thumb (HP:0011304); Pes planus (HP:0001763); Short toe (HP:0001831); Umbilical hernia (HP:0001537); excluded: Trigonocephaly (HP:0000243); excluded: Flat occiput (HP:0005469); excluded: Prominent occiput (HP:0000269); excluded: High forehead (HP:0000348); excluded: Narrow forehead (HP:0000341); excluded: Flat face (HP:0012368); excluded: Round face (HP:0000311); excluded: Broad face (HP:0000283); excluded: Sparse eyebrow (HP:0045075); excluded: Long eyelashes (HP:0000527); excluded: Deeply set eye (HP:0000490); excluded: Proptosis (HP:0000520); excluded: Iris coloboma (HP:0000612); excluded: Cataract (HP:0000518); excluded: Ectopia lentis (HP:0001083); excluded: Esodeviation (HP:0020045); excluded: Long nose (HP:0003189); excluded: Hypoplastic nasal tip (HP:0005278); excluded: Low hanging columella (HP:0009765); excluded: Short philtrum (HP:0000322); excluded: Deep philtrum (HP:0002002); excluded: Thick uppe

In [18]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata)

We output 69 GA4GH phenopackets to the directory phenopackets
